##Load bộ dữ liệu

In [ ]:
!git clone https://github.com/zliucr/CrossNER.git
%cd /content/CrossNER

Cloning into 'CrossNER'...
remote: Enumerating objects: 172, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 172 (delta 2), reused 2 (delta 2), pack-reused 164
Receiving objects: 100% (172/172), 2.30 MiB | 7.28 MiB/s, done.
Resolving deltas: 100% (77/77), done.
/content/CrossNER


##Import modules, thư viện

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from io import BytesIO
import pandas as pd
import numpy as np
import requests
np.random.seed(0)
plt.style.use("ggplot")
import tensorflow as tf

##Lấy các thẻ, ánh xạ dữ liệu,...

In [ ]:
def read_file(in_file_list):
  sentence,wt,word,tag = [],[],[],[]
  for in_file in in_file_list:
    with open(in_file) as f:
        for line in f.readlines():
            sp = line.strip().split('\n\n')
            word_tag = tuple(sp[0].split("\t"))
            if len(word_tag) > 1:
              word.append(word_tag[0])
              tag.append(word_tag[1])
              wt.append(tuple(word_tag))
            else:
              sentence.append(wt)
              wt = []
  word = list(set(word))
  word.append("ENDPAD")
  tag = list(set(tag))
  return sentence,word,tag

In [ ]:
train_m= ["./ner_data/music/train.txt"]
dev_m= ["./ner_data/music/dev.txt"]
test_m= ["./ner_data/music/test.txt"]
train_a= ["./ner_data/ai/train.txt"]
dev_a= ["./ner_data/ai/dev.txt"]
test_a= ["./ner_data/ai/test.txt"]
train_p= ["./ner_data/politics/train.txt"]
dev_p= ["./ner_data/politics/dev.txt"]
test_p= ["./ner_data/politics/test.txt"]
train_l= ["./ner_data/literature/train.txt"]
dev_l= ["./ner_data/literature/dev.txt"]
test_l= ["./ner_data/literature/test.txt"]
train_s= ["./ner_data/science/train.txt"]
dev_s= ["./ner_data/science/dev.txt"]
test_s= ["./ner_data/science/test.txt"]

In [ ]:
def getdata(train,test,dev):
  sentence_tr,words_tr,tags = read_file(train)
  sentence_t,words_t,tags = read_file(test)
  sentence_d,words_d,tags = read_file(dev)
  sentence = sentence_tr + sentence_t + sentence_d
  words = words_tr + words_t + words_d
  num_words = len(words)
  num_tags = len(tags)
  return sentence,words,tags,sentence_tr,words_tr,sentence_t,words_t,sentence_d,words_d,num_words,num_tags
sentence_m,words_m,tags_m,sentence_m_tr,words_m_tr,sentence_m_t,words_m_t,sentence_m_d,words_m_d,num_words_m,num_tags_m = getdata(train_m,test_m,dev_m)
sentence_a,words_a,tags_a,sentence_a_tr,words_a_tr,sentence_a_t,words_a_t,sentence_a_d,words_a_d,num_words_a,num_tags_a = getdata(train_a,test_a,dev_a)
sentence_l,words_l,tags_l,sentence_l_tr,words_l_tr,sentence_l_t,words_l_t,sentence_l_d,words_l_d,num_words_l,num_tags_l = getdata(train_l,test_l,dev_l)
sentence_p,words_p,tags_p,sentence_p_tr,words_p_tr,sentence_p_t,words_p_t,sentence_p_d,words_p_d,num_words_p,num_tags_p = getdata(train_p,test_p,dev_p)
sentence_s,words_s,tags_s,sentence_s_tr,words_s_tr,sentence_s_t,words_s_t,sentence_s_d,words_s_d,num_words_s,num_tags_s = getdata(train_s,test_s,dev_s)

In [ ]:
word2idx_m = {w: i + 1 for i, w in enumerate(words_m)}
word2idx_a = {w: i + 1 for i, w in enumerate(words_a)}
word2idx_p = {w: i + 1 for i, w in enumerate(words_p)}
word2idx_l = {w: i + 1 for i, w in enumerate(words_l)}
word2idx_s = {w: i + 1 for i, w in enumerate(words_s)}
tag2idx_m = {t: i for i, t in enumerate(tags_m)}
tag2idx_a = {t: i for i, t in enumerate(tags_a)}
tag2idx_p = {t: i for i, t in enumerate(tags_p)}
tag2idx_l = {t: i for i, t in enumerate(tags_l)}
tag2idx_s = {t: i for i, t in enumerate(tags_s)}

##Padding

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_len = 80
def ToSequence_d(sentence_d,words_d,max_len,word2idx,tag2idx):
  num_words_d = len(words_d)
  word2idx_d = {w: i + 1 for i, w in enumerate(words_d)}
  x_dev = [[word2idx_d[w[0]] for w in s] for s in sentence_d]
  x_dev = pad_sequences(maxlen=max_len, sequences=x_dev, padding="post", value=num_words_d-1)
  y_dev = [[tag2idx[w[1]] for w in s] for s in sentence_d]
  y_dev = pad_sequences(maxlen=max_len, sequences=y_dev, padding="post", value=tag2idx["O"])
  return x_dev,y_dev
def ToSequence_tr(sentence_tr,words_tr,max_len,word2idx,tag2idx):
  num_words_tr = len(words_tr)
  word2idx_tr = {w: i + 1 for i, w in enumerate(words_tr)}
  x_train = [[word2idx_tr[w[0]] for w in s] for s in sentence_tr]
  x_train = pad_sequences(maxlen=max_len, sequences=x_train, padding="post", value=num_words_tr-1)
  y_train = [[tag2idx[w[1]] for w in s] for s in sentence_tr]
  y_train = pad_sequences(maxlen=max_len, sequences=y_train, padding="post", value=tag2idx["O"])
  return x_train,y_train
def ToSequence_t(sentence_t,words_t,max_len,word2idx,tag2idx):
  num_words_t = len(words_t)
  word2idx_t = {w: i + 1 for i, w in enumerate(words_t)}
  x_test = [[word2idx_t[w[0]] for w in s] for s in sentence_t]
  x_test = pad_sequences(maxlen=max_len, sequences=x_test, padding="post", value=num_words_t-1)
  y_test = [[tag2idx[w[1]] for w in s] for s in sentence_t]
  y_test = pad_sequences(maxlen=max_len, sequences=y_test, padding="post", value=tag2idx["O"])
  return x_test,y_test

In [ ]:
x_dev_a,y_dev_a = ToSequence_d(sentence_a_d,words_a_d,max_len,word2idx_a,tag2idx_a)
x_test_a,y_test_a = ToSequence_t(sentence_a_t,words_a_t,max_len,word2idx_a,tag2idx_a)
x_train_a,y_train_a = ToSequence_tr(sentence_a_tr,words_a_tr,max_len,word2idx_a,tag2idx_a)

In [ ]:
x_dev_l,y_dev_l = ToSequence_d(sentence_l_d,words_l_d,max_len,word2idx_l,tag2idx_l)
x_test_l,y_test_l = ToSequence_t(sentence_l_t,words_l_t,max_len,word2idx_l,tag2idx_l)
x_train_l,y_train_l = ToSequence_tr(sentence_l_tr,words_l_tr,max_len,word2idx_l,tag2idx_l)

In [ ]:
x_dev_m,y_dev_m = ToSequence_d(sentence_m_d,words_m_d,max_len,word2idx_m,tag2idx_m)
x_test_m,y_test_m = ToSequence_t(sentence_m_t,words_m_t,max_len,word2idx_m,tag2idx_m)
x_train_m,y_train_m = ToSequence_tr(sentence_m_tr,words_m_tr,max_len,word2idx_m,tag2idx_m)

In [ ]:
x_dev_p,y_dev_p = ToSequence_d(sentence_p_d,words_p_d,max_len,word2idx_p,tag2idx_p)
x_test_p,y_test_p = ToSequence_t(sentence_p_t,words_p_t,max_len,word2idx_p,tag2idx_p)
x_train_p,y_train_p = ToSequence_tr(sentence_p_tr,words_p_tr,max_len,word2idx_p,tag2idx_p)

In [ ]:
x_dev_s,y_dev_s = ToSequence_d(sentence_s_d,words_s_d,max_len,word2idx_s,tag2idx_s)
x_test_s,y_test_s = ToSequence_t(sentence_s_t,words_s_t,max_len,word2idx_s,tag2idx_s)
x_train_s,y_train_s = ToSequence_tr(sentence_s_tr,words_s_tr,max_len,word2idx_s,tag2idx_s)

##Xây dựng mô hình

In [ ]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense,GRU,Conv1D,Conv2D
from tensorflow.keras.layers import TimeDistributed, SpatialDropout1D, Bidirectional
from tensorflow.keras.optimizers import Adam

In [ ]:
def Bi_LSTM(num_words,num_tags,max_len):
  input_word = Input(shape=(max_len,))
  model = Embedding(input_dim=num_words, output_dim=max_len, input_length=max_len)(input_word)
  model = SpatialDropout1D(0.1)(model)
  model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
  out = TimeDistributed(Dense(num_tags, activation="softmax"))(model)
  model = Model(input_word, out)
  model.compile(optimizer=Adam(lr = 2e-4),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
  model.summary()
  return model
def Bi_GRU(num_words,num_tags,max_len):
  input_word = Input(shape=(max_len,))
  model = Embedding(input_dim=num_words, output_dim=max_len, input_length=max_len)(input_word)
  model = SpatialDropout1D(0.1)(model)
  model = Bidirectional(GRU(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
  out = TimeDistributed(Dense(num_tags, activation="softmax"))(model)
  model = Model(input_word, out)
  model.compile(optimizer=Adam(lr = 2e-4),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
  model.summary()
  return model
def Bi_LSTM_CNN(num_words,num_tags,max_len):
  input_word = Input(shape=(max_len,))
  model = Embedding(input_dim=num_words, output_dim=max_len, input_length=max_len)(input_word)
  model = SpatialDropout1D(0.1)(model)
  model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
  model = Conv1D(200,5,padding = "same",activation = "relu")(model)
  out = TimeDistributed(Dense(num_tags, activation="softmax"))(model)
  model = Model(input_word, out)
  model.compile(optimizer=Adam(lr = 2e-4),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
  model.summary()
  return model
def Bi_GRU_CNN(num_words,num_tags,max_len):
  input_word = Input(shape=(max_len,))
  model = Embedding(input_dim=num_words, output_dim=max_len, input_length=max_len)(input_word)
  model = SpatialDropout1D(0.1)(model)
  model = Bidirectional(GRU(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
  model = Conv1D(200,5,padding = "same",activation = "relu")(model)
  out = TimeDistributed(Dense(num_tags, activation="softmax"))(model)
  model = Model(input_word, out)
  model.compile(optimizer=Adam(lr = 2e-4),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
  model.summary()
  return model

##Hàm tính score

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
def score(model, x_test, y_test,tags):
  y_predict = model.predict(x_test)
  y_predict = np.argmax(y_predict, axis=-1)
  y_predict = y_predict.reshape(1,-1)[0]
  y_true = y_test.reshape(1,-1)[0]
  entity_predict = [tags[i] for i in y_predict]
  entity_true = [tags[i] for i in y_true]
  f1_ma = f1_score(entity_true,entity_predict, average="macro")
  f1_mi= f1_score(entity_true,entity_predict, average="micro")
  print("f1 macro: ", f1_ma)
  print("f1 micro: ", f1_mi)

##Training trên bộ AI

In [ ]:
BiLSTM_a = Bi_LSTM(num_words_a,num_tags_a,max_len)
BiGRU_a = Bi_GRU(num_words_a,num_tags_a,max_len)
BiLSTMCNN_a = Bi_LSTM_CNN(num_words_a,num_tags_a,max_len)
BiGRUCNN_a = Bi_GRU_CNN(num_words_a,num_tags_a,max_len)
history1_a = BiLSTM_a.fit(
    x=x_train_a,
    y=y_train_a,
    validation_data=(x_dev_a,y_dev_a),
    batch_size= 2, 
    epochs=20,
    verbose=1
)

history2_a = BiGRU_a.fit(
    x=x_train_a,
    y=y_train_a,
    validation_data=(x_dev_a,y_dev_a),
    batch_size= 2, 
    epochs=20,
    verbose=1
)
history3_a = BiLSTMCNN_a.fit(
    x=x_train_a,
    y=y_train_a,
    validation_data=(x_dev_a,y_dev_a),
    batch_size= 2, 
    epochs=20,
    verbose=1
)
history4_a = BiGRUCNN_a.fit(
    x=x_train_a,
    y=y_train_a,
    validation_data=(x_dev_a,y_dev_a),
    batch_size= 2, 
    epochs=20,
    verbose=1
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 80)]              0         
                                                                 
 embedding_8 (Embedding)     (None, 80, 80)            640400    
                                                                 
 spatial_dropout1d_8 (Spatia  (None, 80, 80)           0         
 lDropout1D)                                                     
                                                                 
 bidirectional_8 (Bidirectio  (None, 80, 200)          144800    
 nal)                                                            
                                                                 
 time_distributed_8 (TimeDis  (None, 80, 29)           5829      
 tributed)                                                       
                                                           

In [ ]:
BiLSTM_a = Bi_LSTM(num_words_a,num_tags_a,max_len)
history1_a = BiLSTM_a.fit(
    x=x_train_a,
    y=y_train_a,
    validation_data=(x_dev_a,y_dev_a),
    batch_size= 2, 
    epochs=20,
    verbose=1
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 80)]              0         
                                                                 
 embedding_12 (Embedding)    (None, 80, 80)            640400    
                                                                 
 spatial_dropout1d_12 (Spati  (None, 80, 80)           0         
 alDropout1D)                                                    
                                                                 
 bidirectional_12 (Bidirecti  (None, 80, 200)          144800    
 onal)                                                           
                                                                 
 time_distributed_12 (TimeDi  (None, 80, 29)           5829      
 stributed)                                                      
                                                          

In [ ]:
BiLSTMCNN_a = Bi_LSTM_CNN(num_words_a,num_tags_a,max_len)
history3_a = BiLSTMCNN_a.fit(
    x=x_train_a,
    y=y_train_a,
    validation_data=(x_dev_a,y_dev_a),
    batch_size= 2, 
    epochs=20,
    verbose=1
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 80)]              0         
                                                                 
 embedding_13 (Embedding)    (None, 80, 80)            640400    
                                                                 
 spatial_dropout1d_13 (Spati  (None, 80, 80)           0         
 alDropout1D)                                                    
                                                                 
 bidirectional_13 (Bidirecti  (None, 80, 200)          144800    
 onal)                                                           
                                                                 
 conv1d_6 (Conv1D)           (None, 80, 200)           200200    
                                                                 
 time_distributed_13 (TimeDi  (None, 80, 29)           582

##Kết quả trên bộ AI

In [ ]:
score(BiLSTM_a, x_test_a, y_test_a, tags_a)

f1 macro:  0.03268016923392585
f1 micro:  0.888631090487239


In [ ]:
score(BiLSTMCNN_a, x_test_a, y_test_a, tags_a)

f1 macro:  0.03248418768359489
f1 micro:  0.8899941995359629


In [ ]:
score(BiGRU_a, x_test_a, y_test_a, tags_a)

f1 macro:  0.03351954285143432
f1 micro:  0.8689675174013921


In [ ]:
score(BiGRUCNN_a, x_test_a, y_test_a, tags_a)

f1 macro:  0.039038806136109855
f1 micro:  0.8226508120649652


##Training trên bộ Literature

In [ ]:
BiLSTM_l = Bi_LSTM(num_words_l,num_tags_l,max_len)
BiGRU_l = Bi_GRU(num_words_l,num_tags_l,max_len)
BiLSTMCNN_l = Bi_LSTM_CNN(num_words_l,num_tags_l,max_len)
BiGRUCNN_l = Bi_GRU_CNN(num_words_l,num_tags_l,max_len)
history1_l = BiLSTM_l.fit(
    x=x_train_l,
    y=y_train_l,
    validation_data=(x_dev_l,y_dev_l),
    batch_size= 2, 
    epochs=20,
    verbose=1
)

history2_l = BiGRU_l.fit(
    x=x_train_l,
    y=y_train_l,
    validation_data=(x_dev_l,y_dev_l),
    batch_size= 2, 
    epochs=20,
    verbose=1
)
history3_l = BiLSTMCNN_l.fit(
    x=x_train_l,
    y=y_train_l,
    validation_data=(x_dev_l,y_dev_l),
    batch_size= 2, 
    epochs=20,
    verbose=1
)
history4_l = BiGRUCNN_l.fit(
    x=x_train_l,
    y=y_train_l,
    validation_data=(x_dev_l,y_dev_l),
    batch_size= 2, 
    epochs=20,
    verbose=1
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 80)]              0         
                                                                 
 embedding_4 (Embedding)     (None, 80, 80)            853680    
                                                                 
 spatial_dropout1d_4 (Spatia  (None, 80, 80)           0         
 lDropout1D)                                                     
                                                                 
 bidirectional_4 (Bidirectio  (None, 80, 200)          144800    
 nal)                                                            
                                                                 
 time_distributed_4 (TimeDis  (None, 80, 25)           5025      
 tributed)                                                       
                                                           

##Kết quả trên bộ Literature

In [ ]:
score(BiLSTM_l, x_test_l, y_test_l, tags_l)

f1 macro:  0.036690859861865904
f1 micro:  0.8433894230769231


In [ ]:
score(BiLSTMCNN_l, x_test_l, y_test_l, tags_l)

f1 macro:  0.039034238467710064
f1 micro:  0.8079026442307692


In [ ]:
score(BiGRU_l, x_test_l, y_test_l, tags_l)

f1 macro:  0.039387245278541275
f1 micro:  0.7930889423076923


In [ ]:
score(BiGRUCNN_l, x_test_l, y_test_l, tags_l)

f1 macro:  0.04696158065811338
f1 micro:  0.7301983173076924


##Training trên bộ music

In [ ]:
BiLSTM_m = Bi_LSTM(num_words_m,num_tags_m,max_len)
BiGRU_m = Bi_GRU(num_words_m,num_tags_m,max_len)
BiLSTMCNN_m = Bi_LSTM_CNN(num_words_m,num_tags_m,max_len)
BiGRUCNN_m = Bi_GRU_CNN(num_words_m,num_tags_m,max_len)
history1_m = BiLSTM_m.fit(
    x=x_train_m,
    y=y_train_m,
    validation_data=(x_dev_m,y_dev_m),
    batch_size= 2, 
    epochs=20,
    verbose=1
)

history2_m = BiGRU_m.fit(
    x=x_train_m,
    y=y_train_m,
    validation_data=(x_dev_m,y_dev_m),
    batch_size= 2, 
    epochs=20,
    verbose=1
)
history3_m = BiLSTMCNN_m.fit(
    x=x_train_m,
    y=y_train_m,
    validation_data=(x_dev_m,y_dev_m),
    batch_size= 2, 
    epochs=20,
    verbose=1
)
history4_m = BiGRUCNN_m.fit(
    x=x_train_m,
    y=y_train_m,
    validation_data=(x_dev_m,y_dev_m),
    batch_size= 2, 
    epochs=20,
    verbose=1
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 80)]              0         
                                                                 
 embedding_14 (Embedding)    (None, 80, 80)            841600    
                                                                 
 spatial_dropout1d_14 (Spati  (None, 80, 80)           0         
 alDropout1D)                                                    
                                                                 
 bidirectional_14 (Bidirecti  (None, 80, 200)          144800    
 onal)                                                           
                                                                 
 time_distributed_14 (TimeDi  (None, 80, 27)           5427      
 stributed)                                                      
                                                          

In [ ]:
BiLSTM_m = Bi_LSTM(num_words_m,num_tags_m,max_len)
history1_m = BiLSTM_m.fit(
    x=x_train_m,
    y=y_train_m,
    validation_data=(x_dev_m,y_dev_m),
    batch_size= 2, 
    epochs=20,
    verbose=1
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "model_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_23 (InputLayer)       [(None, 80)]              0         
                                                                 
 embedding_22 (Embedding)    (None, 80, 80)            841600    
                                                                 
 spatial_dropout1d_22 (Spati  (None, 80, 80)           0         
 alDropout1D)                                                    
                                                                 
 bidirectional_22 (Bidirecti  (None, 80, 200)          144800    
 onal)                                                           
                                                                 
 time_distributed_22 (TimeDi  (None, 80, 27)           5427      
 stributed)                                                      
                                                          

In [ ]:
BiLSTMCNN_m = Bi_LSTM_CNN(num_words_m,num_tags_m,max_len)
history3_m = BiLSTMCNN_m.fit(
    x=x_train_m,
    y=y_train_m,
    validation_data=(x_dev_m,y_dev_m),
    batch_size= 2, 
    epochs=20,
    verbose=1
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "model_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_20 (InputLayer)       [(None, 80)]              0         
                                                                 
 embedding_19 (Embedding)    (None, 80, 80)            841600    
                                                                 
 spatial_dropout1d_19 (Spati  (None, 80, 80)           0         
 alDropout1D)                                                    
                                                                 
 bidirectional_19 (Bidirecti  (None, 80, 200)          144800    
 onal)                                                           
                                                                 
 conv1d_9 (Conv1D)           (None, 80, 200)           200200    
                                                                 
 time_distributed_19 (TimeDi  (None, 80, 27)           542

In [ ]:
BiGRUCNN_m = Bi_GRU_CNN(num_words_m,num_tags_m,max_len)
history4_m = BiGRUCNN_m.fit(
    x=x_train_m,
    y=y_train_m,
    validation_data=(x_dev_m,y_dev_m),
    batch_size= 2, 
    epochs=20,
    verbose=1
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "model_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None, 80)]              0         
                                                                 
 embedding_21 (Embedding)    (None, 80, 80)            841600    
                                                                 
 spatial_dropout1d_21 (Spati  (None, 80, 80)           0         
 alDropout1D)                                                    
                                                                 
 bidirectional_21 (Bidirecti  (None, 80, 200)          109200    
 onal)                                                           
                                                                 
 conv1d_11 (Conv1D)          (None, 80, 200)           200200    
                                                                 
 time_distributed_21 (TimeDi  (None, 80, 27)           542

##Kết quả trên bộ Music

In [ ]:
score(BiLSTM_m, x_test_m, y_test_m, tags_m)

f1 macro:  0.026928310691548565
f1 micro:  0.5652419354838709


In [ ]:
score(BiLSTMCNN_m, x_test_m, y_test_m, tags_m)

f1 macro:  0.02862507197929371
f1 micro:  0.6277419354838709


In [ ]:
score(BiGRU_m, x_test_m, y_test_m, tags_m)

f1 macro:  0.03473995186181903
f1 micro:  0.7295967741935484


In [ ]:
score(BiGRUCNN_m, x_test_m, y_test_m, tags_m)

f1 macro:  0.036202589142181614
f1 micro:  0.7344354838709677


##Training trên bộ Politics

In [ ]:
BiLSTM_p = Bi_LSTM(num_words_p,num_tags_p,max_len)
BiGRU_p = Bi_GRU(num_words_p,num_tags_p,max_len)
BiLSTMCNN_p = Bi_LSTM_CNN(num_words_p,num_tags_p,max_len)
BiGRUCNN_p = Bi_GRU_CNN(num_words_p,num_tags_p,max_len)
history1_p = BiLSTM_p.fit(
    x=x_train_p,
    y=y_train_p,
    validation_data=(x_dev_p,y_dev_p),
    batch_size= 2, 
    epochs=20,
    verbose=1
)

history2_p = BiGRU_p.fit(
    x=x_train_p,
    y=y_train_p,
    validation_data=(x_dev_p,y_dev_p),
    batch_size= 2, 
    epochs=20,
    verbose=1
)
history3_p = BiLSTMCNN_p.fit(
    x=x_train_p,
    y=y_train_p,
    validation_data=(x_dev_p,y_dev_p),
    batch_size= 2, 
    epochs=20,
    verbose=1
)
history4_p = BiGRUCNN_p.fit(
    x=x_train_p,
    y=y_train_p,
    validation_data=(x_dev_p,y_dev_p),
    batch_size= 2, 
    epochs=20,
    verbose=1
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "model_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_24 (InputLayer)       [(None, 80)]              0         
                                                                 
 embedding_23 (Embedding)    (None, 80, 80)            1062880   
                                                                 
 spatial_dropout1d_23 (Spati  (None, 80, 80)           0         
 alDropout1D)                                                    
                                                                 
 bidirectional_23 (Bidirecti  (None, 80, 200)          144800    
 onal)                                                           
                                                                 
 time_distributed_23 (TimeDi  (None, 80, 19)           3819      
 stributed)                                                      
                                                          

In [ ]:
BiLSTM_p = Bi_LSTM(num_words_p,num_tags_p,max_len)
history1_p = BiLSTM_p.fit(
    x=x_train_p,
    y=y_train_p,
    validation_data=(x_dev_p,y_dev_p),
    batch_size= 2, 
    epochs=20,
    verbose=1
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "model_35"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_36 (InputLayer)       [(None, 80)]              0         
                                                                 
 embedding_35 (Embedding)    (None, 80, 80)            1062880   
                                                                 
 spatial_dropout1d_35 (Spati  (None, 80, 80)           0         
 alDropout1D)                                                    
                                                                 
 bidirectional_35 (Bidirecti  (None, 80, 200)          144800    
 onal)                                                           
                                                                 
 time_distributed_35 (TimeDi  (None, 80, 19)           3819      
 stributed)                                                      
                                                          

In [ ]:
BiLSTMCNN_p = Bi_LSTM_CNN(num_words_p,num_tags_p,max_len)
history3_p = BiLSTMCNN_p.fit(
    x=x_train_p,
    y=y_train_p,
    validation_data=(x_dev_p,y_dev_p),
    batch_size= 2, 
    epochs=20,
    verbose=1
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "model_38"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_39 (InputLayer)       [(None, 80)]              0         
                                                                 
 embedding_38 (Embedding)    (None, 80, 80)            1062880   
                                                                 
 spatial_dropout1d_38 (Spati  (None, 80, 80)           0         
 alDropout1D)                                                    
                                                                 
 bidirectional_38 (Bidirecti  (None, 80, 200)          144800    
 onal)                                                           
                                                                 
 conv1d_21 (Conv1D)          (None, 80, 200)           200200    
                                                                 
 time_distributed_38 (TimeDi  (None, 80, 19)           381

##Kết quả trên bộ Politics

In [ ]:
score(BiLSTM_p, x_test_p, y_test_p, tags_p)

f1 macro:  0.04745343996603576
f1 micro:  0.6715437788018433


In [ ]:
score(BiLSTMCNN_p, x_test_p, y_test_p, tags_p)

f1 macro:  0.05288663770126474
f1 micro:  0.6860791090629801


In [ ]:
score(BiGRU_p, x_test_p, y_test_p, tags_p)

f1 macro:  0.057569563108303065
f1 micro:  0.7146697388632871


In [ ]:
score(BiGRUCNN_p, x_test_p, y_test_p, tags_p)

f1 macro:  0.05876218927544976
f1 micro:  0.6883448540706605


##Training trên bộ Science

In [ ]:
BiLSTM_s = Bi_LSTM(num_words_s,num_tags_s,max_len)
BiGRU_s = Bi_GRU(num_words_s,num_tags_s,max_len)
BiLSTMCNN_s = Bi_LSTM_CNN(num_words_s,num_tags_s,max_len)
BiGRUCNN_s = Bi_GRU_CNN(num_words_s,num_tags_s,max_len)
history1_s = BiLSTM_s.fit(
    x=x_train_s,
    y=y_train_s,
    validation_data=(x_dev_s, y_dev_s),
    batch_size= 2, 
    epochs=20,
    verbose=1
)

history2_s = BiGRU_s.fit(
    x=x_train_s,
    y=y_train_s,
    validation_data=(x_dev_s,y_dev_s),
    batch_size= 2, 
    epochs=20,
    verbose=1
)
history3_s = BiLSTMCNN_s.fit(
    x=x_train_s,
    y=y_train_s,
    validation_data=(x_dev_s,y_dev_s),
    batch_size= 2, 
    epochs=20,
    verbose=1
)
history4_s = BiGRUCNN_s.fit(
    x=x_train_s,
    y=y_train_s,
    validation_data=(x_dev_s,y_dev_s),
    batch_size= 2, 
    epochs=20,
    verbose=1
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "model_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_29 (InputLayer)       [(None, 80)]              0         
                                                                 
 embedding_28 (Embedding)    (None, 80, 80)            1034480   
                                                                 
 spatial_dropout1d_28 (Spati  (None, 80, 80)           0         
 alDropout1D)                                                    
                                                                 
 bidirectional_28 (Bidirecti  (None, 80, 200)          144800    
 onal)                                                           
                                                                 
 time_distributed_28 (TimeDi  (None, 80, 35)           7035      
 stributed)                                                      
                                                          

In [ ]:
BiLSTMCNN_s = Bi_LSTM_CNN(num_words_s,num_tags_s,max_len)
history3_s = BiLSTMCNN_s.fit(
    x=x_train_s,
    y=y_train_s,
    validation_data=(x_dev_s,y_dev_s),
    batch_size= 2, 
    epochs=20,
    verbose=1
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "model_39"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_40 (InputLayer)       [(None, 80)]              0         
                                                                 
 embedding_39 (Embedding)    (None, 80, 80)            1034480   
                                                                 
 spatial_dropout1d_39 (Spati  (None, 80, 80)           0         
 alDropout1D)                                                    
                                                                 
 bidirectional_39 (Bidirecti  (None, 80, 200)          144800    
 onal)                                                           
                                                                 
 conv1d_22 (Conv1D)          (None, 80, 200)           200200    
                                                                 
 time_distributed_39 (TimeDi  (None, 80, 35)           703

In [ ]:
BiGRUCNN_s = Bi_GRU_CNN(num_words_s,num_tags_s,max_len)
history4_s = BiGRUCNN_s.fit(
    x=x_train_s,
    y=y_train_s,
    validation_data=(x_dev_s,y_dev_s),
    batch_size= 2, 
    epochs=20,
    verbose=1
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "model_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_35 (InputLayer)       [(None, 80)]              0         
                                                                 
 embedding_34 (Embedding)    (None, 80, 80)            1034480   
                                                                 
 spatial_dropout1d_34 (Spati  (None, 80, 80)           0         
 alDropout1D)                                                    
                                                                 
 bidirectional_34 (Bidirecti  (None, 80, 200)          109200    
 onal)                                                           
                                                                 
 conv1d_18 (Conv1D)          (None, 80, 200)           200200    
                                                                 
 time_distributed_34 (TimeDi  (None, 80, 35)           703

##Kết quả trên bộ Science

In [ ]:
score(BiLSTM_s, x_test_s, y_test_s, tags_s)

f1 macro:  0.02373903553357686
f1 micro:  0.639963167587477


In [ ]:
score(BiLSTMCNN_s, x_test_s, y_test_s, tags_s)

f1 macro:  0.02899753560546264
f1 micro:  0.7094843462246776


In [ ]:
score(BiGRU_s, x_test_s, y_test_s, tags_s)

f1 macro:  0.030335574132316787
f1 micro:  0.7319060773480663


In [ ]:
score(BiGRUCNN_s, x_test_s, y_test_s, tags_s)

f1 macro:  0.03344970740660151
f1 micro:  0.7467311233885819


##Kết quả độ đo trên các mô hình

In [ ]:
from sklearn.metrics import classification_report
def report(model, x_test, y_test,tags):
  y_predict = model.predict(x_test)
  y_predict = np.argmax(y_predict, axis=-1)
  y_predict = y_predict.reshape(1,-1)[0]
  y_true = y_test.reshape(1,-1)[0]
  entity_predict = [tags[i] for i in y_predict]
  entity_true = [tags[i] for i in y_true]
  report = classification_report(entity_true,entity_predict)
  return report

In [ ]:
rp1 = report(BiLSTM_a, x_test_a, y_test_a, tags_a)
print(rp1)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                precision    recall  f1-score   support

   B-algorithm       0.00      0.00      0.00       177
  B-conference       0.00      0.00      0.00        93
     B-country       0.00      0.00      0.00        44
       B-field       0.00      0.00      0.00       207
    B-location       0.00      0.00      0.00        39
     B-metrics       0.00      0.00      0.00       191
        B-misc       0.00      0.00      0.00       181
B-organisation       0.00      0.00      0.00       145
      B-person       0.00      0.00      0.00        67
     B-product       0.00      0.00      0.00       198
 B-programlang       0.00      0.00      0.00        60
  B-researcher       0.00      0.00      0.00       160
        B-task       0.00      0.00      0.00       219
  B-university       0.00      0.00      0.00        28
   I-algorithm       0.00      0.00      0.00       198
  I-conference       0.00      0.00      0.00       217
     I-country       0.00      0.00      0.00  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
rp1 = report(BiLSTMCNN_a, x_test_a, y_test_a, tags_a)
print(rp1)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                precision    recall  f1-score   support

   B-algorithm       0.00      0.00      0.00       177
  B-conference       0.00      0.00      0.00        93
     B-country       0.00      0.00      0.00        44
       B-field       0.00      0.00      0.00       207
    B-location       0.00      0.00      0.00        39
     B-metrics       0.00      0.00      0.00       191
        B-misc       0.00      0.00      0.00       181
B-organisation       0.00      0.00      0.00       145
      B-person       0.00      0.00      0.00        67
     B-product       0.00      0.00      0.00       198
 B-programlang       0.00      0.00      0.00        60
  B-researcher       0.00      0.00      0.00       160
        B-task       0.00      0.00      0.00       219
  B-university       0.00      0.00      0.00        28
   I-algorithm       0.00      0.00      0.00       198
  I-conference       0.00      0.00      0.00       217
     I-country       0.00      0.00      0.00  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
rp1 = report(BiGRU_a, x_test_a, y_test_a, tags_a)
print(rp1)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                precision    recall  f1-score   support

   B-algorithm       0.00      0.00      0.00       177
  B-conference       0.00      0.00      0.00        93
     B-country       0.00      0.00      0.00        44
       B-field       0.00      0.00      0.00       207
    B-location       0.00      0.00      0.00        39
     B-metrics       0.00      0.00      0.00       191
        B-misc       0.00      0.00      0.00       181
B-organisation       0.00      0.00      0.00       145
      B-person       0.00      0.00      0.00        67
     B-product       0.00      0.00      0.00       198
 B-programlang       0.00      0.00      0.00        60
  B-researcher       0.00      0.00      0.00       160
        B-task       0.00      0.00      0.00       219
  B-university       0.00      0.00      0.00        28
   I-algorithm       0.02      0.12      0.04       198
  I-conference       0.00      0.00      0.00       217
     I-country       0.00      0.00      0.00  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
rp1 = report(BiGRUCNN_a, x_test_a, y_test_a, tags_a)
print(rp1)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                precision    recall  f1-score   support

   B-algorithm       0.00      0.00      0.00       177
  B-conference       0.00      0.00      0.00        93
     B-country       0.00      0.00      0.00        44
       B-field       0.00      0.00      0.00       207
    B-location       0.00      0.00      0.00        39
     B-metrics       0.00      0.00      0.00       191
        B-misc       0.05      0.04      0.04       181
B-organisation       0.00      0.00      0.00       145
      B-person       0.00      0.00      0.00        67
     B-product       0.05      0.01      0.01       198
 B-programlang       0.00      0.00      0.00        60
  B-researcher       0.00      0.00      0.00       160
        B-task       0.02      0.02      0.02       219
  B-university       0.00      0.00      0.00        28
   I-algorithm       0.01      0.02      0.01       198
  I-conference       0.01      0.05      0.02       217
     I-country       0.00      0.00      0.00  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
rp1 = report(BiLSTM_l, x_test_l, y_test_l, tags_l)
print(rp1)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                 precision    recall  f1-score   support

        B-award       0.00      0.00      0.00       141
         B-book       0.00      0.00      0.00       418
      B-country       0.00      0.00      0.00       101
        B-event       0.00      0.00      0.00        45
B-literarygenre       0.00      0.00      0.00       194
     B-location       0.00      0.00      0.00        99
     B-magazine       0.00      0.00      0.00        57
         B-misc       0.00      0.00      0.00       239
 B-organisation       0.00      0.00      0.00       110
       B-person       0.00      0.00      0.00       174
         B-poem       0.00      0.00      0.00       120
       B-writer       0.00      0.00      0.00       567
        I-award       0.00      0.00      0.00       335
         I-book       0.25      0.00      0.00       898
      I-country       0.00      0.00      0.00        56
        I-event       0.00      0.00      0.00        89
I-literarygenre       0.00    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
rp1 = report(BiLSTMCNN_l, x_test_l, y_test_l, tags_l)
print(rp1)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                 precision    recall  f1-score   support

        B-award       0.00      0.00      0.00       141
         B-book       0.00      0.00      0.00       418
      B-country       0.00      0.00      0.00       101
        B-event       0.00      0.00      0.00        45
B-literarygenre       0.00      0.00      0.00       194
     B-location       0.00      0.00      0.00        99
     B-magazine       0.00      0.00      0.00        57
         B-misc       0.00      0.00      0.00       239
 B-organisation       0.00      0.00      0.00       110
       B-person       0.00      0.00      0.00       174
         B-poem       0.00      0.00      0.00       120
       B-writer       0.00      0.00      0.00       567
        I-award       0.00      0.00      0.00       335
         I-book       0.05      0.11      0.07       898
      I-country       0.00      0.00      0.00        56
        I-event       0.00      0.00      0.00        89
I-literarygenre       0.00    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
rp1 = report(BiGRU_l, x_test_l, y_test_l, tags_l)
print(rp1)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                 precision    recall  f1-score   support

        B-award       0.00      0.00      0.00       141
         B-book       0.00      0.00      0.00       418
      B-country       0.00      0.00      0.00       101
        B-event       0.00      0.00      0.00        45
B-literarygenre       0.00      0.00      0.00       194
     B-location       0.00      0.00      0.00        99
     B-magazine       0.00      0.00      0.00        57
         B-misc       0.00      0.00      0.00       239
 B-organisation       0.00      0.00      0.00       110
       B-person       0.00      0.00      0.00       174
         B-poem       0.00      0.00      0.00       120
       B-writer       0.00      0.00      0.00       567
        I-award       0.00      0.00      0.00       335
         I-book       0.05      0.17      0.08       898
      I-country       0.00      0.00      0.00        56
        I-event       0.00      0.00      0.00        89
I-literarygenre       0.00    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
rp1 = report(BiGRUCNN_l, x_test_l, y_test_l, tags_l)
print(rp1)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                 precision    recall  f1-score   support

        B-award       0.03      0.02      0.02       141
         B-book       0.02      0.01      0.02       418
      B-country       0.00      0.00      0.00       101
        B-event       0.00      0.00      0.00        45
B-literarygenre       0.05      0.01      0.01       194
     B-location       0.00      0.00      0.00        99
     B-magazine       0.00      0.00      0.00        57
         B-misc       0.05      0.02      0.03       239
 B-organisation       0.00      0.00      0.00       110
       B-person       0.00      0.00      0.00       174
         B-poem       0.01      0.03      0.01       120
       B-writer       0.04      0.06      0.05       567
        I-award       0.05      0.12      0.07       335
         I-book       0.04      0.05      0.04       898
      I-country       0.00      0.00      0.00        56
        I-event       0.00      0.00      0.00        89
I-literarygenre       0.00    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
rp1 = report(BiLSTM_m, x_test_m, y_test_m, tags_m)
print(rp1)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                     precision    recall  f1-score   support

            B-album       0.00      0.00      0.00       332
            B-award       0.00      0.00      0.00       260
             B-band       0.00      0.01      0.00       462
          B-country       0.00      0.00      0.00       160
            B-event       0.00      0.00      0.00        69
         B-location       0.00      0.00      0.00       343
             B-misc       0.00      0.00      0.00       171
    B-musicalartist       0.00      0.00      0.00       515
B-musicalinstrument       0.00      0.00      0.00        22
       B-musicgenre       0.00      0.00      0.00       486
     B-organisation       0.00      0.00      0.00       208
           B-person       0.00      0.00      0.00        80
             B-song       0.00      0.00      0.00       227
            I-album       0.00      0.00      0.00       675
            I-award       0.00      0.00      0.00       837
             I-band    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
rp1 = report(BiLSTMCNN_m, x_test_m, y_test_m, tags_m)
print(rp1)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                     precision    recall  f1-score   support

            B-album       0.00      0.00      0.00       332
            B-award       0.00      0.00      0.00       260
             B-band       0.00      0.00      0.00       462
          B-country       0.00      0.00      0.00       160
            B-event       0.00      0.00      0.00        69
         B-location       0.00      0.00      0.00       343
             B-misc       0.00      0.00      0.00       171
    B-musicalartist       0.00      0.00      0.00       515
B-musicalinstrument       0.00      0.00      0.00        22
       B-musicgenre       0.00      0.00      0.00       486
     B-organisation       0.00      0.00      0.00       208
           B-person       0.00      0.00      0.00        80
             B-song       0.00      0.00      0.00       227
            I-album       0.00      0.00      0.00       675
            I-award       0.00      0.00      0.00       837
             I-band    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
rp1 = report(BiGRU_m, x_test_m, y_test_m, tags_m)
print(rp1)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                     precision    recall  f1-score   support

            B-album       0.00      0.00      0.00       332
            B-award       0.00      0.00      0.00       260
             B-band       0.01      0.00      0.00       462
          B-country       0.00      0.00      0.00       160
            B-event       0.00      0.00      0.00        69
         B-location       0.00      0.00      0.00       343
             B-misc       0.00      0.00      0.00       171
    B-musicalartist       0.00      0.00      0.00       515
B-musicalinstrument       0.00      0.00      0.00        22
       B-musicgenre       0.00      0.00      0.00       486
     B-organisation       0.00      0.00      0.00       208
           B-person       0.00      0.00      0.00        80
             B-song       0.00      0.00      0.00       227
            I-album       0.00      0.00      0.00       675
            I-award       0.04      0.21      0.07       837
             I-band    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
rp1 = report(BiGRUCNN_m, x_test_m, y_test_m, tags_m)
print(rp1)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                     precision    recall  f1-score   support

            B-album       0.00      0.00      0.00       332
            B-award       0.00      0.00      0.00       260
             B-band       0.00      0.00      0.00       462
          B-country       0.00      0.00      0.00       160
            B-event       0.00      0.00      0.00        69
         B-location       0.00      0.00      0.00       343
             B-misc       0.00      0.00      0.00       171
    B-musicalartist       0.12      0.01      0.01       515
B-musicalinstrument       0.00      0.00      0.00        22
       B-musicgenre       0.00      0.00      0.00       486
     B-organisation       0.00      0.00      0.00       208
           B-person       0.00      0.00      0.00        80
             B-song       0.00      0.00      0.00       227
            I-album       0.04      0.08      0.06       675
            I-award       0.03      0.06      0.04       837
             I-band    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
rp1 = report(BiLSTM_p, x_test_p, y_test_p, tags_p)
print(rp1)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                  precision    recall  f1-score   support

       B-country       0.00      0.00      0.00       418
      B-election       0.00      0.00      0.00       434
         B-event       0.00      0.00      0.00       195
      B-location       0.00      0.00      0.00       599
          B-misc       0.00      0.00      0.00       258
  B-organisation       0.00      0.00      0.00       513
        B-person       0.00      0.00      0.00       354
B-politicalparty       0.00      0.00      0.00       953
    B-politician       0.03      0.03      0.03       485
       I-country       0.00      0.00      0.00       219
      I-election       0.02      0.05      0.03      1640
         I-event       0.00      0.00      0.00       464
      I-location       0.00      0.00      0.00       189
          I-misc       0.00      0.00      0.00       130
  I-organisation       0.06      0.00      0.00      1011
        I-person       0.00      0.00      0.00       338
I-politicalpa

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
rp1 = report(BiLSTMCNN_p, x_test_p, y_test_p, tags_p)
print(rp1)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                  precision    recall  f1-score   support

       B-country       0.00      0.00      0.00       418
      B-election       0.00      0.00      0.00       434
         B-event       0.00      0.00      0.00       195
      B-location       0.00      0.00      0.00       599
          B-misc       0.00      0.00      0.00       258
  B-organisation       0.00      0.00      0.00       513
        B-person       0.00      0.00      0.00       354
B-politicalparty       0.03      0.00      0.00       953
    B-politician       0.05      0.04      0.04       485
       I-country       0.00      0.00      0.00       219
      I-election       0.05      0.25      0.08      1640
         I-event       0.01      0.01      0.01       464
      I-location       0.00      0.00      0.00       189
          I-misc       0.00      0.00      0.00       130
  I-organisation       0.00      0.00      0.00      1011
        I-person       0.00      0.00      0.00       338
I-politicalpa

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
rp1 = report(BiGRU_p, x_test_p, y_test_p, tags_p)
print(rp1)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                  precision    recall  f1-score   support

       B-country       0.00      0.00      0.00       418
      B-election       0.00      0.00      0.00       434
         B-event       0.00      0.00      0.00       195
      B-location       0.02      0.01      0.01       599
          B-misc       0.00      0.00      0.00       258
  B-organisation       0.03      0.01      0.01       513
        B-person       0.00      0.00      0.00       354
B-politicalparty       0.13      0.04      0.06       953
    B-politician       0.03      0.07      0.04       485
       I-country       0.00      0.00      0.00       219
      I-election       0.02      0.04      0.02      1640
         I-event       0.00      0.00      0.00       464
      I-location       0.00      0.00      0.00       189
          I-misc       0.00      0.00      0.00       130
  I-organisation       0.03      0.04      0.04      1011
        I-person       0.00      0.00      0.00       338
I-politicalpa

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
rp1 = report(BiGRUCNN_p, x_test_p, y_test_p, tags_p)
print(rp1)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                  precision    recall  f1-score   support

       B-country       0.00      0.00      0.00       418
      B-election       0.01      0.00      0.01       434
         B-event       0.00      0.00      0.00       195
      B-location       0.03      0.01      0.01       599
          B-misc       0.03      0.00      0.01       258
  B-organisation       0.02      0.02      0.02       513
        B-person       0.00      0.00      0.00       354
B-politicalparty       0.05      0.01      0.02       953
    B-politician       0.03      0.07      0.05       485
       I-country       0.00      0.00      0.00       219
      I-election       0.01      0.02      0.02      1640
         I-event       0.00      0.00      0.00       464
      I-location       0.01      0.02      0.02       189
          I-misc       0.00      0.00      0.00       130
  I-organisation       0.04      0.13      0.06      1011
        I-person       0.00      0.00      0.00       338
I-politicalpa

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
rp1 = report(BiLSTM_s, x_test_s, y_test_s, tags_s)
print(rp1)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                      precision    recall  f1-score   support

   B-academicjournal       0.00      0.00      0.00       180
B-astronomicalobject       0.00      0.01      0.00       337
             B-award       0.00      0.00      0.00       146
  B-chemicalcompound       0.00      0.00      0.00       160
   B-chemicalelement       0.00      0.00      0.00        85
           B-country       0.00      0.00      0.00        27
        B-discipline       0.00      0.00      0.00        73
            B-enzyme       0.00      0.00      0.00        80
             B-event       0.00      0.00      0.00        31
          B-location       0.00      0.00      0.00       170
              B-misc       0.02      0.01      0.01       520
      B-organisation       0.00      0.00      0.00       273
            B-person       0.00      0.00      0.00       230
           B-protein       0.00      0.00      0.00       156
         B-scientist       0.03      0.00      0.01       471
       

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
rp1 = report(BiLSTMCNN_s, x_test_s, y_test_s, tags_s)
print(rp1)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                      precision    recall  f1-score   support

   B-academicjournal       0.00      0.00      0.00       180
B-astronomicalobject       0.00      0.00      0.00       337
             B-award       0.00      0.00      0.00       146
  B-chemicalcompound       0.00      0.00      0.00       160
   B-chemicalelement       0.00      0.00      0.00        85
           B-country       0.00      0.00      0.00        27
        B-discipline       0.00      0.00      0.00        73
            B-enzyme       0.00      0.00      0.00        80
             B-event       0.00      0.00      0.00        31
          B-location       0.00      0.00      0.00       170
              B-misc       0.00      0.00      0.00       520
      B-organisation       0.01      0.01      0.01       273
            B-person       0.01      0.00      0.01       230
           B-protein       0.00      0.00      0.00       156
         B-scientist       0.03      0.05      0.04       471
       

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
rp1 = report(BiGRU_s, x_test_s, y_test_s, tags_s)
print(rp1)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                      precision    recall  f1-score   support

   B-academicjournal       0.00      0.00      0.00       180
B-astronomicalobject       0.00      0.00      0.00       337
             B-award       0.00      0.00      0.00       146
  B-chemicalcompound       0.00      0.00      0.00       160
   B-chemicalelement       0.00      0.00      0.00        85
           B-country       0.00      0.00      0.00        27
        B-discipline       0.00      0.00      0.00        73
            B-enzyme       0.00      0.00      0.00        80
             B-event       0.00      0.00      0.00        31
          B-location       0.00      0.00      0.00       170
              B-misc       0.03      0.02      0.03       520
      B-organisation       0.02      0.06      0.02       273
            B-person       0.12      0.00      0.01       230
           B-protein       0.00      0.00      0.00       156
         B-scientist       0.01      0.01      0.01       471
       

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
rp1 = report(BiGRUCNN_s, x_test_s, y_test_s, tags_s)
print(rp1)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                      precision    recall  f1-score   support

   B-academicjournal       0.00      0.00      0.00       180
B-astronomicalobject       0.05      0.01      0.02       337
             B-award       0.00      0.00      0.00       146
  B-chemicalcompound       0.00      0.00      0.00       160
   B-chemicalelement       0.00      0.00      0.00        85
           B-country       0.00      0.00      0.00        27
        B-discipline       0.00      0.00      0.00        73
            B-enzyme       0.09      0.03      0.04        80
             B-event       0.00      0.00      0.00        31
          B-location       0.01      0.01      0.01       170
              B-misc       0.03      0.03      0.03       520
      B-organisation       0.01      0.01      0.01       273
            B-person       0.03      0.01      0.01       230
           B-protein       0.00      0.00      0.00       156
         B-scientist       0.05      0.07      0.06       471
       

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##Kết quả dự đoán

In [ ]:
def predict(model, x_test, y_test, words, tags):
  i = np.random.randint(0, x_test.shape[0])
  p = model.predict(np.array([x_test[i]]))
  p = np.argmax(p, axis=-1)
  y_true = y_test[i]
  print("{:15}{:5}\t {}\n".format("Word", "True", "Pred"))
  print("-" *30)
  for w, true, pred in zip(x_test[i], y_true, p[0]):
    print("{:15}{}\t{}".format(words[w-1], tags[true], tags[pred]))

In [ ]:
predict(BiLSTM_a, x_test_a, y_test_a, words_a, tags_a)